In [1]:
import torch,os,time
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
from utils.data import RandomNoiseGenerator,Data
from utils.train_history import train_history
from utils import visualizer,util
import itertools
from models.model import Generator, Discriminator
# from models.model import Generator_test, Discriminator_test
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
if torch.backends.cudnn.enabled:
    torch.backends.cudnn.benchmark = True
device

device(type='cuda', index=3)

In [3]:
latent_size = 512
target_resol = 256
first_resol = 4
use_sigmoid = False
train_kimg = 600
train_img = train_kimg*1000
transition_kimg = 600
transition_img = transition_kimg*1000

g_lr_max =  0.0002
d_lr_max =  0.0002

beta1 = 0.5
beta2 = 0.999

lambda_A=10
lambda_B=10
lambda_recon=0.8
lambda_idt=0

report_it = 100
show_it = 100
save_it=100

In [4]:
display_id=1
display_winsize=256
display_ncols=4
display_server='http://localhost'
display_port=8097
display_env='pggan'
results_dir='results'
project_name = 'pggan1'
project_dir=os.path.join(results_dir,project_name)

vis = visualizer.Visualizer(display_id,display_winsize,display_ncols,display_server,display_port,display_env,
                 project_name,results_dir)

if not os.path.isdir(project_dir):
    os.makedirs(project_dir)

create web directory results/pggan1/web...


In [ ]:
real_dir = '/data/persona_cyclegan/real/trainA'
anime_dir = '/data/persona_cyclegan/anime/trainB'
# real_dir = '/data/persona_cyclegan/real_test'
# anime_dir = '/data/persona_cyclegan/anime_test'
data_A = Data(real_dir)
data_B = Data(anime_dir)

In [ ]:
G_A = Generator(num_channels=3, latent_size=latent_size, resolution=target_resol, fmap_max=latent_size, fmap_base=8192, tanh_at_end=True)
G_B = Generator(num_channels=3, latent_size=latent_size, resolution=target_resol, fmap_max=latent_size, fmap_base=8192, tanh_at_end=True)
# G_A = Generator_test()
# G_B = Generator_test()

D_A = Discriminator(num_channels=3, mbstat_avg='all', resolution=target_resol, fmap_max=latent_size, fmap_base=8192, sigmoid_at_end=True)
D_B = Discriminator(num_channels=3, mbstat_avg='all', resolution=target_resol, fmap_max=latent_size, fmap_base=8192, sigmoid_at_end=True)
# D_A = Discriminator_test()
# D_B = Discriminator_test()

print(G_A)
print(D_A)
G_A,G_B,D_A,D_B = G_A.to(device),G_B.to(device),D_A.to(device),D_B.to(device)
optim_G = optim.Adam(itertools.chain(G_A.parameters(),G_B.parameters()), g_lr_max, betas=(beta1, beta2))
optim_D_A = optim.Adam(D_A.parameters(), d_lr_max, betas=(beta1, beta2))
optim_D_B = optim.Adam(D_B.parameters(), d_lr_max, betas=(beta1, beta2))

all_models = {'G_A.pkl':G_A,
             'G_B.pkl':G_B,
             'D_A.pkl':D_A,
             'D_B.pkl':D_B}
            

print('---------- Networks initialized -------------')
for model_name,model in [('G',G_A),('D',D_A)]:
    num_params = 0
    for param in model.parameters():
        num_params += param.numel()
    print(str.format('{} has {} number of parameters', model_name, num_params))
print('-----------------------------------------------')

Generator(
  (output_layer): GSelectLayer(
    (pre): ModuleList(
      (0): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): Conv2d(3, 512, kernel_size=(7, 7), stride=(1, 1))
        (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace)
      )
      (1): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): Conv2d(3, 512, kernel_size=(7, 7), stride=(1, 1))
        (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace)
      )
      (2): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): Conv2d(3, 512, kernel_size=(7, 7), stride=(1, 1))
        (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace)
      )
      (3): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): Conv2d(3, 512, kernel_size=(7, 7), stride=(1, 1))
        (2): BatchNorm2d(

---------- Networks initialized -------------
G has 45463253 number of parameters
D has 19212353 number of parameters
-----------------------------------------------


In [ ]:
rampup_kimg = 10000
rampdown_kimg = 10000
total_kimg = 10000

def _rampup(epoch, rampup_length):
    if epoch < rampup_length:
        p = max(0.0, float(epoch)) / float(rampup_length)
        p = 1.0 - p
        return np.exp(-p*p*5.0)
    else:
        return 1.0

def _rampdown_linear(epoch, num_epochs, rampdown_length):
    if epoch >= num_epochs - rampdown_length:
        return float(num_epochs - epoch) / rampdown_length
    else:
        return 1.0

In [ ]:
def get_bs(resolution):
    R = int(np.log2(resolution))
    if R < 7:
        bs = 32 / 2**(max(0, R-4))
    else:
        bs = 8 / 2**(min(2, R-7))
    return int(bs)

bs_map = {2**R: get_bs(2**R) for R in range(2, 11)}

In [ ]:
train_hist = train_history(['G_gan_loss',
                              'G_idt_loss',
                              'G_cycle_loss',
                              'D_A_real_loss',
                              'D_A_fake_loss',
                              'D_B_real_loss',                                         
                              'D_B_fake_loss',                                         
                              ])

In [ ]:
L1_loss = nn.L1Loss().to(device) 
GAN_loss = nn.MSELoss().to(device)

In [ ]:
def save_models(models, folder):
    for k, v in models.items():
        torch.save(v.state_dict(), os.path.join(folder, k))

## Train

In [ ]:
to_level = int(np.log2(target_resol))
from_level = int(np.log2(first_resol))

for R in range(from_level-1, to_level):
    batch_size = bs_map[2 ** (R+1)]
    phases = {'stabilize':[0, train_img//batch_size], 'fade_in':[train_img//batch_size+1, (transition_img+train_img)//batch_size]}
    if R==to_level-1:
        del phases['fade_in']
    print('starting scale %d, batch_size is %d, and the iterations in each phase is %d'%(R,batch_size,train_img//batch_size))
#     for phase in ['fade_in']: 
    for phase in ['stabilize', 'fade_in']:
        num_pool = 80
        fake_A_pool = util.ImagePool(num_pool)
        fake_B_pool = util.ImagePool(num_pool)
        if phase in phases:
            _range = phases[phase]
            from_it = _range[0]
            cur_it = 0
            total_it = _range[1]
            remaining_it = total_it-from_it
            cur_nimg = _range[0]*batch_size
            resol = 2 ** (R+1)
            
            previous_time = time.time()
            phase_start_time = time.time()
            
            for it in range(from_it, total_it):
                cur_it += 1
                if phase == 'stabilize':
                    cur_level = R
                else:
                    cur_level = R + cur_it/remaining_it
                cur_resol = 2 ** int(np.ceil(cur_level+1))

                # get a batch noise and real images
                real_A = data_A.next(batch_size,cur_resol,cur_level)
                real_A = real_A.to(device)
                
                real_B = data_B.next(batch_size,cur_resol,cur_level)
                real_B = real_B.to(device)
                # ===preprocess===
#                 for param_group in optim_G.param_groups:
#                     lrate_coef = _rampup(cur_nimg / 1000.0, rampup_kimg)
#                     lrate_coef *= _rampdown_linear(cur_nimg / 1000.0,total_kimg, rampdown_kimg)
#                     param_group['lr'] = lrate_coef * g_lr_max
  
#                 for param_group in optim_D_A.param_groups:
#                     lrate_coef = _rampup(cur_nimg / 1000.0, rampup_kimg)
#                     lrate_coef *= _rampdown_linear(cur_nimg / 1000.0, total_kimg, rampdown_kimg)
#                     param_group['lr'] = lrate_coef * d_lr_max
#                 for param_group in optim_D_B.param_groups:
#                     lrate_coef = _rampup(cur_nimg / 1000.0, rampup_kimg)
#                     lrate_coef *= _rampdown_linear(cur_nimg / 1000.0, total_kimg, rampdown_kimg)
#                     param_group['lr'] = lrate_coef * d_lr_max
                    
                
                # ===update G===
                for model in [D_A,D_B]:
                    for param in model.parameters():
                        param.requires_grad = False
                        
                optim_G.zero_grad()
                
                #G_A
                fake_B = G_A(real_A, cur_level)
                d_fake_A = D_B(fake_B, cur_level)
                d_fake_A_loss = GAN_loss(d_fake_A, torch.ones(d_fake_A.size(), device=device))
                
                sim_A_loss = L1_loss(fake_B,real_A) * lambda_idt
                
                recon_A = G_B(fake_B,cur_level)
                recon_A_loss = L1_loss(recon_A,real_A) * lambda_recon
                
                G_A_loss = (d_fake_A_loss+sim_A_loss+recon_A_loss)*lambda_A
                G_A_loss.backward()
                
                fake_A = G_B(real_B,cur_level)
                d_fake_B = D_A(fake_A,cur_level)
                d_fake_B_loss = GAN_loss(d_fake_B, torch.ones(d_fake_B.size(), device=device))

                sim_B_loss = L1_loss(fake_A,real_B)*lambda_idt
                
                recon_B = G_A(fake_A,cur_level)
                recon_B_loss = L1_loss(recon_B,real_B)*lambda_recon
                
                G_B_loss = (d_fake_B_loss + sim_B_loss + recon_B_loss)*lambda_B
                G_B_loss.backward()
                
                G_gan_loss = d_fake_A_loss + d_fake_B_loss
                G_idt_loss = sim_A_loss + sim_B_loss
                G_recon_loss = recon_A_loss + recon_B_loss

                optim_G.step()
 
#                 ===update D===
                for model in [D_A,D_B]:
                    for param in model.parameters():
                        param.requires_grad = True
                #D_B
                optim_D_B.zero_grad()

                fake_B = G_A(real_A, cur_level=cur_level)
                fake_B_from_pool = fake_B_pool.query(fake_B.detach())
                d_real_B = D_B(real_B, cur_level=cur_level)
                d_fake_B = D_B(fake_B_from_pool, cur_level=cur_level)

#                 d_real_B_loss = torch.mean((d_real_B-1)**2)
#                 d_fake_B_loss = torch.mean((d_fake_B-0)**2)
#                 d_real_B_loss = GAN_loss(d_real_B, torch.ones(d_real_B.size(), device=device))
#                 d_fake_B_loss = GAN_loss(d_fake_B, torch.zeros(d_fake_B.size(), device=device))
                d_real_B_loss = GAN_loss(d_real_B, 1-torch.rand(d_real_B.size(), device=device)/10.0)
                d_fake_B_loss = GAN_loss(d_fake_B, torch.rand(d_real_B.size(), device=device)/10.0)        
        
        
                d_loss_B = 0.5 * (d_real_B_loss + d_fake_B_loss)
                d_loss_B.backward()
                optim_D_B.step()
                
                #D_A
                optim_D_A.zero_grad()
 
                fake_A = G_B(real_B, cur_level=cur_level)
                fake_A_from_pool = fake_A_pool.query(fake_A.detach())
                d_real_A = D_A(real_A, cur_level=cur_level)
                d_fake_A = D_A(fake_A_from_pool, cur_level=cur_level)

#                 d_real_A_loss = torch.mean((d_real_A-1)**2)
#                 d_fake_A_loss = torch.mean((d_fake_A-0)**2)
#                 d_real_A_loss = GAN_loss(d_real_A, torch.ones(d_real_A.size(), device=device))
#                 d_fake_A_loss = GAN_loss(d_fake_A, torch.zeros(d_fake_A.size(), device=device))             
                d_real_A_loss = GAN_loss(d_real_A, 1-torch.rand(d_real_B.size(), device=device)/10.0)
                d_fake_A_loss = GAN_loss(d_fake_A, torch.rand(d_real_B.size(), device=device)/10.0)      
    
                d_loss_A = 0.5 * (d_real_A_loss + d_fake_A_loss)
                d_loss_A.backward()    
                optim_D_A.step()
                
                cur_nimg += batch_size
                
                cur_scale = R+cur_it/total_it/2
                if phase == 'fade_in':
                    cur_scale+=0.5
                
#                 if it%50 == 0:
#                     print('d_real_A_loss:%.3f,d_fake_A_loss:%.3f'%(d_real_A_loss,d_fake_A_loss))
#                     print(d_real_B)
#                     print(d_fake_B)

                # ===report ===
                train_hist.add_params([G_gan_loss,G_idt_loss,G_recon_loss,d_real_A_loss,d_fake_A_loss,d_real_B_loss,d_fake_B_loss])
                if it% report_it == 0:                    
                    phase_time = time.time() - phase_start_time
                    losses = train_hist.check_current_avg()
                    it_time = time.time() - previous_time
                    vis.print_current_losses(cur_level, it, losses, it_time, phase_time)
                    vis.plot_current_losses(cur_scale, losses)
                    previous_time = time.time()
                # ===generate sample images===
                if it % show_it == 0:
                    save_result = it % save_it == 0
                    vis.display_current_results([real_A,fake_B,recon_A],cur_scale,save_result)
                    
                # ===save model===
                if it % save_it == 0:
                    save_models(all_models,project_dir)
                    
#                 break

    model_folder_at_scale = os.path.join(project_dir,str(R))
    if not os.path.isdir(model_folder_at_scale):
        os.makedirs(model_folder_at_scale)
    save_models(all_models,model_folder_at_scale)

starting scale 1, batch_size is 32, and the iterations in each phase is 18750
(current_level: 1.000, iters: 0, time_from_last_report: 0.643, since_the_phase_start: 0.642) G_gan_loss: 2.779 G_idt_loss: 0.000 G_cycle_loss: 1.397 D_A_real_loss: 1.314 D_A_fake_loss: 0.160 D_B_real_loss: 1.253 D_B_fake_loss: 0.107 
(current_level: 1.000, iters: 100, time_from_last_report: 24.531, since_the_phase_start: 25.147) G_gan_loss: 1.066 G_idt_loss: 0.000 G_cycle_loss: 1.042 D_A_real_loss: 0.253 D_A_fake_loss: 0.120 D_B_real_loss: 0.208 D_B_fake_loss: 0.096 
(current_level: 1.000, iters: 200, time_from_last_report: 23.565, since_the_phase_start: 48.720) G_gan_loss: 1.011 G_idt_loss: 0.000 G_cycle_loss: 0.920 D_A_real_loss: 0.164 D_A_fake_loss: 0.145 D_B_real_loss: 0.044 D_B_fake_loss: 0.042 
(current_level: 1.000, iters: 300, time_from_last_report: 24.932, since_the_phase_start: 73.659) G_gan_loss: 1.101 G_idt_loss: 0.000 G_cycle_loss: 0.784 D_A_real_loss: 0.155 D_A_fake_loss: 0.143 D_B_real_loss: 0.

(current_level: 1.000, iters: 3400, time_from_last_report: 23.815, since_the_phase_start: 809.987) G_gan_loss: 1.338 G_idt_loss: 0.000 G_cycle_loss: 1.148 D_A_real_loss: 0.081 D_A_fake_loss: 0.104 D_B_real_loss: 0.002 D_B_fake_loss: 0.002 
(current_level: 1.000, iters: 3500, time_from_last_report: 23.594, since_the_phase_start: 833.589) G_gan_loss: 1.425 G_idt_loss: 0.000 G_cycle_loss: 1.125 D_A_real_loss: 0.082 D_A_fake_loss: 0.109 D_B_real_loss: 0.002 D_B_fake_loss: 0.002 
(current_level: 1.000, iters: 3600, time_from_last_report: 23.582, since_the_phase_start: 857.180) G_gan_loss: 1.409 G_idt_loss: 0.000 G_cycle_loss: 1.119 D_A_real_loss: 0.075 D_A_fake_loss: 0.094 D_B_real_loss: 0.002 D_B_fake_loss: 0.002 
(current_level: 1.000, iters: 3700, time_from_last_report: 23.579, since_the_phase_start: 880.767) G_gan_loss: 1.448 G_idt_loss: 0.000 G_cycle_loss: 1.132 D_A_real_loss: 0.093 D_A_fake_loss: 0.101 D_B_real_loss: 0.002 D_B_fake_loss: 0.002 
(current_level: 1.000, iters: 3800, time

(current_level: 1.000, iters: 6900, time_from_last_report: 23.734, since_the_phase_start: 1642.703) G_gan_loss: 1.318 G_idt_loss: 0.000 G_cycle_loss: 1.125 D_A_real_loss: 0.066 D_A_fake_loss: 0.089 D_B_real_loss: 0.002 D_B_fake_loss: 0.001 
(current_level: 1.000, iters: 7000, time_from_last_report: 23.927, since_the_phase_start: 1666.641) G_gan_loss: 1.400 G_idt_loss: 0.000 G_cycle_loss: 1.117 D_A_real_loss: 0.082 D_A_fake_loss: 0.109 D_B_real_loss: 0.001 D_B_fake_loss: 0.001 
(current_level: 1.000, iters: 7100, time_from_last_report: 23.890, since_the_phase_start: 1690.542) G_gan_loss: 1.349 G_idt_loss: 0.000 G_cycle_loss: 1.118 D_A_real_loss: 0.062 D_A_fake_loss: 0.105 D_B_real_loss: 0.002 D_B_fake_loss: 0.001 
(current_level: 1.000, iters: 7200, time_from_last_report: 23.664, since_the_phase_start: 1714.218) G_gan_loss: 1.486 G_idt_loss: 0.000 G_cycle_loss: 1.114 D_A_real_loss: 0.064 D_A_fake_loss: 0.099 D_B_real_loss: 0.009 D_B_fake_loss: 0.016 
(current_level: 1.000, iters: 7300, 

(current_level: 1.000, iters: 10300, time_from_last_report: 23.704, since_the_phase_start: 2454.841) G_gan_loss: 1.459 G_idt_loss: 0.000 G_cycle_loss: 1.110 D_A_real_loss: 0.065 D_A_fake_loss: 0.103 D_B_real_loss: 0.001 D_B_fake_loss: 0.001 
(current_level: 1.000, iters: 10400, time_from_last_report: 23.682, since_the_phase_start: 2478.535) G_gan_loss: 1.386 G_idt_loss: 0.000 G_cycle_loss: 1.119 D_A_real_loss: 0.081 D_A_fake_loss: 0.103 D_B_real_loss: 0.002 D_B_fake_loss: 0.001 
(current_level: 1.000, iters: 10500, time_from_last_report: 23.695, since_the_phase_start: 2502.243) G_gan_loss: 1.318 G_idt_loss: 0.000 G_cycle_loss: 1.122 D_A_real_loss: 0.068 D_A_fake_loss: 0.104 D_B_real_loss: 0.001 D_B_fake_loss: 0.001 
(current_level: 1.000, iters: 10600, time_from_last_report: 23.980, since_the_phase_start: 2526.234) G_gan_loss: 1.304 G_idt_loss: 0.000 G_cycle_loss: 1.117 D_A_real_loss: 0.064 D_A_fake_loss: 0.083 D_B_real_loss: 0.008 D_B_fake_loss: 0.010 
(current_level: 1.000, iters: 10

(current_level: 1.000, iters: 13700, time_from_last_report: 23.932, since_the_phase_start: 3265.923) G_gan_loss: 1.368 G_idt_loss: 0.000 G_cycle_loss: 1.130 D_A_real_loss: 0.072 D_A_fake_loss: 0.107 D_B_real_loss: 0.001 D_B_fake_loss: 0.001 
(current_level: 1.000, iters: 13800, time_from_last_report: 23.740, since_the_phase_start: 3289.676) G_gan_loss: 1.726 G_idt_loss: 0.000 G_cycle_loss: 1.132 D_A_real_loss: 0.013 D_A_fake_loss: 0.045 D_B_real_loss: 0.001 D_B_fake_loss: 0.001 
(current_level: 1.000, iters: 13900, time_from_last_report: 23.689, since_the_phase_start: 3313.383) G_gan_loss: 1.395 G_idt_loss: 0.000 G_cycle_loss: 1.106 D_A_real_loss: 0.066 D_A_fake_loss: 0.128 D_B_real_loss: 0.001 D_B_fake_loss: 0.001 
(current_level: 1.000, iters: 14000, time_from_last_report: 23.929, since_the_phase_start: 3337.327) G_gan_loss: 1.416 G_idt_loss: 0.000 G_cycle_loss: 1.103 D_A_real_loss: 0.096 D_A_fake_loss: 0.135 D_B_real_loss: 0.001 D_B_fake_loss: 0.001 
(current_level: 1.000, iters: 14

...

(current_level: 3.184, iters: 22200, time_from_last_report: 104.185, since_the_phase_start: 3590.485) G_gan_loss: 1.224 G_idt_loss: 0.000 G_cycle_loss: 0.610 D_A_real_loss: 0.030 D_A_fake_loss: 0.022 D_B_real_loss: 0.003 D_B_fake_loss: 0.003 
(current_level: 3.189, iters: 22300, time_from_last_report: 104.293, since_the_phase_start: 3694.851) G_gan_loss: 1.453 G_idt_loss: 0.000 G_cycle_loss: 0.608 D_A_real_loss: 0.013 D_A_fake_loss: 0.015 D_B_real_loss: 0.002 D_B_fake_loss: 0.002 
(current_level: 3.195, iters: 22400, time_from_last_report: 103.831, since_the_phase_start: 3798.754) G_gan_loss: 1.317 G_idt_loss: 0.000 G_cycle_loss: 0.597 D_A_real_loss: 0.010 D_A_fake_loss: 0.011 D_B_real_loss: 0.048 D_B_fake_loss: 0.021 
(current_level: 3.200, iters: 22500, time_from_last_report: 104.433, since_the_phase_start: 3903.256) G_gan_loss: 1.331 G_idt_loss: 0.000 G_cycle_loss: 0.599 D_A_real_loss: 0.004 D_A_fake_loss: 0.006 D_B_real_loss: 0.050 D_B_fake_loss: 0.028 
(current_level: 3.205, iters

(current_level: 3.365, iters: 25600, time_from_last_report: 103.783, since_the_phase_start: 7126.577) G_gan_loss: 1.637 G_idt_loss: 0.000 G_cycle_loss: 0.588 D_A_real_loss: 0.005 D_A_fake_loss: 0.009 D_B_real_loss: 0.002 D_B_fake_loss: 0.002 
(current_level: 3.371, iters: 25700, time_from_last_report: 104.380, since_the_phase_start: 7231.025) G_gan_loss: 1.635 G_idt_loss: 0.000 G_cycle_loss: 0.588 D_A_real_loss: 0.010 D_A_fake_loss: 0.007 D_B_real_loss: 0.030 D_B_fake_loss: 0.020 
(current_level: 3.376, iters: 25800, time_from_last_report: 103.822, since_the_phase_start: 7334.924) G_gan_loss: 1.751 G_idt_loss: 0.000 G_cycle_loss: 0.555 D_A_real_loss: 0.002 D_A_fake_loss: 0.001 D_B_real_loss: 0.007 D_B_fake_loss: 0.009 
(current_level: 3.381, iters: 25900, time_from_last_report: 103.938, since_the_phase_start: 7438.934) G_gan_loss: 1.647 G_idt_loss: 0.000 G_cycle_loss: 0.566 D_A_real_loss: 0.002 D_A_fake_loss: 0.005 D_B_real_loss: 0.007 D_B_fake_loss: 0.012 
(current_level: 3.387, iters

(current_level: 3.547, iters: 29000, time_from_last_report: 104.046, since_the_phase_start: 10668.012) G_gan_loss: 1.772 G_idt_loss: 0.000 G_cycle_loss: 0.515 D_A_real_loss: 0.002 D_A_fake_loss: 0.002 D_B_real_loss: 0.006 D_B_fake_loss: 0.015 
(current_level: 3.552, iters: 29100, time_from_last_report: 104.139, since_the_phase_start: 10772.230) G_gan_loss: 1.586 G_idt_loss: 0.000 G_cycle_loss: 0.527 D_A_real_loss: 0.013 D_A_fake_loss: 0.009 D_B_real_loss: 0.002 D_B_fake_loss: 0.001 
(current_level: 3.557, iters: 29200, time_from_last_report: 103.958, since_the_phase_start: 10876.264) G_gan_loss: 1.768 G_idt_loss: 0.000 G_cycle_loss: 0.521 D_A_real_loss: 0.010 D_A_fake_loss: 0.024 D_B_real_loss: 0.001 D_B_fake_loss: 0.001 
(current_level: 3.563, iters: 29300, time_from_last_report: 103.674, since_the_phase_start: 10980.008) G_gan_loss: 1.789 G_idt_loss: 0.000 G_cycle_loss: 0.512 D_A_real_loss: 0.007 D_A_fake_loss: 0.007 D_B_real_loss: 0.002 D_B_fake_loss: 0.002 
(current_level: 3.568, i

(current_level: 3.728, iters: 32400, time_from_last_report: 103.772, since_the_phase_start: 14202.614) G_gan_loss: 0.653 G_idt_loss: 0.000 G_cycle_loss: 0.619 D_A_real_loss: 0.019 D_A_fake_loss: 0.033 D_B_real_loss: 0.037 D_B_fake_loss: 0.050 
(current_level: 3.733, iters: 32500, time_from_last_report: 103.886, since_the_phase_start: 14306.581) G_gan_loss: 0.607 G_idt_loss: 0.000 G_cycle_loss: 0.653 D_A_real_loss: 0.031 D_A_fake_loss: 0.051 D_B_real_loss: 0.040 D_B_fake_loss: 0.051 
(current_level: 3.739, iters: 32600, time_from_last_report: 105.038, since_the_phase_start: 14411.691) G_gan_loss: 0.689 G_idt_loss: 0.000 G_cycle_loss: 0.594 D_A_real_loss: 0.027 D_A_fake_loss: 0.044 D_B_real_loss: 0.028 D_B_fake_loss: 0.042 
(current_level: 3.744, iters: 32700, time_from_last_report: 103.680, since_the_phase_start: 14515.442) G_gan_loss: 0.619 G_idt_loss: 0.000 G_cycle_loss: 0.604 D_A_real_loss: 0.022 D_A_fake_loss: 0.036 D_B_real_loss: 0.036 D_B_fake_loss: 0.052 
(current_level: 3.749, i

(current_level: 3.909, iters: 35800, time_from_last_report: 104.150, since_the_phase_start: 17734.944) G_gan_loss: 1.209 G_idt_loss: 0.000 G_cycle_loss: 0.625 D_A_real_loss: 0.026 D_A_fake_loss: 0.044 D_B_real_loss: 0.003 D_B_fake_loss: 0.002 
(current_level: 3.915, iters: 35900, time_from_last_report: 103.955, since_the_phase_start: 17838.970) G_gan_loss: 1.223 G_idt_loss: 0.000 G_cycle_loss: 0.603 D_A_real_loss: 0.029 D_A_fake_loss: 0.044 D_B_real_loss: 0.002 D_B_fake_loss: 0.002 
(current_level: 3.920, iters: 36000, time_from_last_report: 103.898, since_the_phase_start: 17942.943) G_gan_loss: 0.992 G_idt_loss: 0.000 G_cycle_loss: 0.676 D_A_real_loss: 0.036 D_A_fake_loss: 0.060 D_B_real_loss: 0.014 D_B_fake_loss: 0.013 
(current_level: 3.925, iters: 36100, time_from_last_report: 103.977, since_the_phase_start: 18046.990) G_gan_loss: 1.085 G_idt_loss: 0.000 G_cycle_loss: 0.660 D_A_real_loss: 0.030 D_A_fake_loss: 0.072 D_B_real_loss: 0.005 D_B_fake_loss: 0.006 
(current_level: 3.931, i